In [1]:
#written by Zilin Shen and Daniel de Mello
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import time
import numpy as np
from packet_capturer_daniel import PacketCapturer
from window_manager import WindowManager
from feature_extractor import FeatureExtractor
from numpy_generator import NumpyGenerator
from lstm import LSTM
from label_packet import generate_label_data, label_packets
from random_perturb import random_perturb
from pso import PSO
from lstm import Lstm


# the hyperparameters of window manager
window_size = 1
move_size = 0.1 #how long the movement of window
swnd = True
# the class of detector
# attack, infection,reconnaissance
kind = "reconnaissance"
# the chosen algorithm "feedforward" or "lstm"
algorithm = "lstm"

# -----------get the preprocessed training and testing saved as .npy files
test_label_infection = np.load('preprocessed/test_label_infection.npy')
train_label_infection = np.load('preprocessed/train_label_infection.npy')
test_data_infection = np.load('preprocessed/test_data_infection.npy')
train_data_infection = np.load('preprocessed/train_data_infection.npy')

test_label_reconnaissance = np.load('preprocessed/test_label_reconnaissance.npy')
train_label_reconnaissance = np.load('preprocessed/train_label_reconnaissance.npy')
test_data_reconnaissance = np.load('preprocessed/test_data_reconnaissance.npy')
train_data_reconnaissance = np.load('preprocessed/train_data_reconnaissance.npy')

test_label_attack = np.load('preprocessed/test_label_attack.npy')
train_label_attack = np.load('preprocessed/train_label_attack.npy')
test_data_attack = np.load('preprocessed/test_data_attack.npy')
train_data_attack = np.load('preprocessed/train_data_attack.npy')

all_data = {"infection": 
                        {
                        'train': [train_data_infection, train_label_infection], 
                        'test': [test_data_infection, test_label_infection]
                        },
            "attack": 
                        {
                        'train': [train_data_attack, train_label_attack], 
                        'test': [test_data_attack, test_label_attack]
                        },
            "reconnaissance": 
                        {
                        'train': [train_data_reconnaissance, train_label_reconnaissance], 
                        'test': [test_data_reconnaissance, test_label_reconnaissance]
                        }
            }

test_label_infection_unb = np.load('preprocessed/test_label_infection_unbalanced.npy')
train_label_infection_unb = np.load('preprocessed/train_label_infection_unbalanced.npy')
test_data_infection_unb = np.load('preprocessed/test_data_infection_unbalanced.npy')
train_data_infection_unb = np.load('preprocessed/train_data_infection_unbalanced.npy')

infection_unb_data = {'train': [train_data_infection_unb, train_label_infection_unb],
                      'test': [test_data_infection_unb, test_label_infection_unb],
                     }
# -----------------start learning with some algorithm----------------------

def print_header(string):
    bar_len = len(string)
    bar = '-'*bar_len
    print(bar+ '\n' + string + '\n' + bar)

def train_detector(detector, data, kind):

    #train data
    train_data = data['train']
    train_examples = train_data[0]
    train_labels = train_data[1]

    #test data
    test_data = data['test']
    test_examples = test_data[0]
    test_labels = test_data[1]
        
    features_len = train_examples.shape[1]
    print('features len is ', features_len)
    
    print_header("Training {} detector".format(kind))
    detector.learning(features_len, train_examples, train_labels)
                    
    print_header("Measureing {} detector performance on test data".format(kind))
    detector.detection(test_examples, test_labels)
    return detector 



2023-04-26 01:57:04.265293: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/oracle/instantclient_19_8:/opt/oracle/instantclient_19_8:/p/cuda-11.5
2023-04-26 01:57:04.265319: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


the count of label lines 20084
(20084,)


In [2]:
all_data['infection']['train'][1].sum()/len(all_data['infection']['train'][1])

0.5

In [3]:
infection_unb_data['train'][0].shape


(10016, 41)

In [4]:
attack_detector = train_detector(Lstm("lstm-attack"), all_data['attack'], 'attack')
reconnaissance_detector = train_detector(Lstm("lstm-recon"), all_data['reconnaissance'], 'reconnaissance')
infection_detector = train_detector(Lstm("lstm-infection"), all_data['infection'], 'infection')

features len is  41
------------------------
Training attack detector
------------------------


2023-04-26 01:57:07.170223: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-26 01:57:07.170888: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-26 01:57:07.762557: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 715.34GiB/s
2023-04-26 01:57:07.762650: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/oracle/instantclient_19_8:/opt/oracle/instantclient_19_8:/p/cuda-11.5
2023-04-26 01:57:07.762699: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 

the info of dataset, (19152, 1, 41)
shuffle is enabled for training
Epoch 1/50
539/539 - 4s - loss: 0.1347 - accuracy: 0.9560 - auc: 0.9894 - recall: 0.9682 - precision: 0.9451 - val_loss: 0.0777 - val_accuracy: 0.9703 - val_auc: 0.9955 - val_recall: 0.9843 - val_precision: 0.9573
Epoch 2/50
539/539 - 1s - loss: 0.0645 - accuracy: 0.9762 - auc: 0.9965 - recall: 0.9871 - precision: 0.9659 - val_loss: 0.0638 - val_accuracy: 0.9786 - val_auc: 0.9969 - val_recall: 0.9927 - val_precision: 0.9654
Epoch 3/50
539/539 - 1s - loss: 0.0565 - accuracy: 0.9800 - auc: 0.9972 - recall: 0.9901 - precision: 0.9705 - val_loss: 0.0598 - val_accuracy: 0.9755 - val_auc: 0.9970 - val_recall: 0.9864 - val_precision: 0.9652
Epoch 4/50
539/539 - 1s - loss: 0.0551 - accuracy: 0.9806 - auc: 0.9973 - recall: 0.9904 - precision: 0.9713 - val_loss: 0.0570 - val_accuracy: 0.9770 - val_auc: 0.9971 - val_recall: 0.9937 - val_precision: 0.9615
Epoch 5/50
539/539 - 1s - loss: 0.0505 - accuracy: 0.9819 - auc: 0.9979 - re

In [ ]:
train_events_per_detector = []
for detector in  [attack_detector, infection_detector, reconnaissance_detector]:
    #examples_train = infection_unb_data['train'][0]
    examples_train = all_data['infection']['train'][0]
    train_events = detector.predict(examples_train)
    print('proportion of events tagged by {} detector: '.format(detector.get_name()), np.sum(np.array(train_events)>0.5)/len(train_events))
    train_events_per_detector.append(train_events)

benign_events = []
no_events = len(train_events_per_detector[0])
for i  in range(no_events):
    not_attack = 1-train_events_per_detector[0][i]
    not_infection = 1-train_events_per_detector[1][i]
    not_recon = 1-train_events_per_detector[2][i]
    benign_events.append(not_attack * not_infection * not_recon)
train_events_per_detector.append(benign_events)
print('proportion of events estimated to be benign : ', np.sum(np.array(benign_events)>0.5)/len(benign_events))
train_events_per_detector = np.array(train_events_per_detector).squeeze()





proportion of events tagged by lstm-attack detector:  0.08381269552931678
proportion of events tagged by lstm-infection detector:  0.4244626097487133
proportion of events tagged by lstm-recon detector:  0.47910990009082655
proportion of events estimated to be benign :  0.06983550307800988


In [ ]:
def retrain_detector(detector, retrain_data, retrain_labels, test_data, test_labels):

        
    features_len = retrain_data.shape[1]
    print('features len is ', features_len)
    
    print_header("Training {} detector".format(kind))
    detector.learning(features_len, retrain_data, retrain_labels)
                    
    print_header("Measureing {} detector performance on test data".format(kind))
    detector.detection(test_data, test_labels)
    return detector 

In [ ]:
test_data = all_data['infection'][]

In [6]:
def softmax(x):
    """
    Compute the softmax of vector x.
    """
    # Subtract the maximum value of x from each element of x to avoid overflow
    # and make the softmax numerically stable.
    z = x - np.max(x)
    
    # Calculate the exponential of each element in z.
    exp_z = np.exp(z)
    
    # Calculate the sum of the exponential values.
    sum_exp_z = np.sum(exp_z)
    
    # Divide each element of the exponential vector by the sum of exponential values.
    softmax_z = exp_z / sum_exp_z
    
    return softmax_z

train_events_softmax = []
for i in range(no_events):
    softmax_event = softmax(train_events_per_detector[:,i])
    train_events_softmax.append(softmax_event)


In [7]:
np.sum(np.argmax(train_events_softmax, axis=-1)==0)/no_events

0.02901402765162983

In [8]:
train_events_softmax

[array([0.17437206, 0.47399244, 0.17726341, 0.17437206], dtype=float32),
 array([0.1748777 , 0.1748777 , 0.1748777 , 0.47536692], dtype=float32),
 array([0.36490425, 0.36487645, 0.1351068 , 0.13511246], dtype=float32),
 array([0.1761863 , 0.46613538, 0.1766717 , 0.18100657], dtype=float32),
 array([0.17458251, 0.43878472, 0.19980086, 0.18683192], dtype=float32),
 array([0.1770406 , 0.4660595 , 0.17608485, 0.180815  ], dtype=float32),
 array([0.17487767, 0.1748778 , 0.4753668 , 0.17487767], dtype=float32),
 array([0.17841302, 0.2221075 , 0.38644293, 0.21303648], dtype=float32),
 array([0.17841302, 0.2221075 , 0.38644293, 0.21303648], dtype=float32),
 array([0.17841302, 0.2221075 , 0.38644293, 0.21303648], dtype=float32),
 array([0.17488067, 0.17488067, 0.47534817, 0.17489053], dtype=float32),
 array([0.17841302, 0.2221075 , 0.38644293, 0.21303648], dtype=float32),
 array([0.17841302, 0.2221075 , 0.38644293, 0.21303648], dtype=float32),
 array([0.17841302, 0.2221075 , 0.38644293, 0.21303

In [12]:
from perm_seq2seq_attention import PermSeq2seqAttention

In [14]:
seq2seq = PermSeq2seqAttention('seq2seq')           

In [15]:
train_events_softmax[4]

array([0.17299663, 0.42224696, 0.21670878, 0.18804763], dtype=float32)

In [16]:
config = {"sequence_length": 10, 
          "permute_truncated": True,
          "use_prob_embedding": False,
          "rv": 1
        }

In [17]:
train_label_infection_unb.sum()

107.0

In [18]:
#seq2seq.learning(train_events_softmax, train_label_infection_unb, config)
seq2seq.learning(train_events_softmax, all_data['infection']['train'][1], config)

Permute truncated is True
Use prob embedding is False
Permute Truncated is enabled
truncated idxs are [[ 0  1  2  3  4  5  6  7  8  9]
 [ 1  2  3  4  5  6  7  8  9 10]
 [ 2  3  4  5  6  7  8  9 10 11]
 [ 3  4  5  6  7  8  9 10 11 12]
 [ 4  5  6  7  8  9 10 11 12 13]
 [ 5  6  7  8  9 10 11 12 13 14]
 [ 6  7  8  9 10 11 12 13 14 15]
 [ 7  8  9 10 11 12 13 14 15 16]
 [ 8  9 10 11 12 13 14 15 16 17]
 [ 9 10 11 12 13 14 15 16 17 18]
 [10 11 12 13 14 15 16 17 18 19]
 [11 12 13 14 15 16 17 18 19 20]
 [12 13 14 15 16 17 18 19 20 21]
 [13 14 15 16 17 18 19 20 21 22]
 [14 15 16 17 18 19 20 21 22 23]
 [15 16 17 18 19 20 21 22 23 24]
 [16 17 18 19 20 21 22 23 24 25]
 [17 18 19 20 21 22 23 24 25 26]
 [18 19 20 21 22 23 24 25 26 27]
 [19 20 21 22 23 24 25 26 27 28]]
perm truncated idxs are [[ 0  1  4  2  3  5  8  6  7  9]
 [ 1  2  3  5  4  6  9 10  7  8]
 [ 4  5  2  3  6  9  7 11 10  8]
 [ 5  4  3 10  7  9  6 12  8 11]
 [ 5  4  8  9 10 11  6  7 12 13]
 [ 5  6 11  9  7  8 12 10 13 14]
 [11 12  8 10  

In [20]:
res_probs, res_idxs = seq2seq.analysis(train_events_softmax, all_data['infection']['train'][1], config)

Permute truncated is True
Use prob embedding is False


In [22]:

res_idxs

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 856,
 857,
 858,
 859,
 860,
 861,
 862,
 863,
 864,
 865,
 866,
 867,
 868,
 869,
 870,
 871,
 872,
 873,
 1802,
 1803,
 1804,
 1805,
 1806,
 1807,
 1808,
 1809,
 1810,
 1811,
 1812,
 1813,
 1814,
 1815,
 1816,
 1817,
 1818,
 2751,
 2752,
 2753,
 2754,
 2755,
 2756,
 2757,
 2758,
 2759,
 2760,
 3696,
 3697,
 3698,
 3699,
 3700,
 3701,
 3702,
 3703,
 6356,
 6357,
 8136,
 8137,
 8138,
 8139,
 8140,
 8141,
 8142,
 8143,
 8144,
 8145,
 8146,
 8147,
 8148,
 8149,
 8150,
 8151,
 8152,
 8153,
 8154,
 8155,
 8156,
 8157,
 8158,
 8159,
 8160,
 8161,
 8162,
 8163,
 8164,
 8165,
 8166,
 8167,
 8168,
 8169,
 8170,
 8171,
 8172,
 8173,
 8174,
 8175,
 8176,
 8177,
 8178,
 8179,
 8180,
 8181,
 8182,
 8183,
 8184,
 8185,
 8186,
 8187,
 8188,
 8189,
 8190,
 8191,
 8192,
 8193,
 8194,
 8195,
 8196,
 8197,
 8198,
 8199,
 8256,
 8257,
 8258,
 8259,
 8260,
 8261,
 8262,
 8263,
 8264,
 8265,
 8266,
 8267,
 8268,
 8269,
 8270,
 8271,
 8272,
 8273,
 8276,
 8277,
 8278

In [30]:
ret

118471717.0

In [5]:
PermSeq2seqAttention

perm_seq2seq_attention.PermSeq2seqAttention

In [16]:
np.array(train_events_per_detector).shape

(3, 10016, 1, 1)

In [1]:
len(all_data['reconnaissance']['train'][1])

NameError: name 'all_data' is not defined

In [7]:
np.sum(all_data['reconnaissance']['train'][1]==0)

2811

In [8]:
np.sum(all_data['reconnaissance']['train'][1]==1)/len(all_data['reconnaissance']['train'][0])

0.7193490415335463

In [9]:
np.sum(all_data['attack']['train'][1]==1)/len(all_data['attack']['train'][0])

0.0439297124600639

In [10]:
np.sum(all_data['infection']['train'][1]==1)/len(all_data['infection']['train'][0])

0.010682907348242811

In [61]:
def retrain_detector(detector, retrain_data, retrain_labels, test_data, test_labels):

        
    features_len = retrain_data.shape[1]
    print('features len is ', features_len)
    
    print_header("Training {} detector".format(kind))
    detector.learning(features_len, retrain_data, retrain_labels)
                    
    print_header("Measureing {} detector performance on test data".format(kind))
    detector.detection(test_data, test_labels)
    return detector 
